In [3]:
from dotenv import load_dotenv
load_dotenv()
import logging
from bs4 import BeautifulSoup
import html2text
import httpx
import yaml
import json
from pydantic import Field, BaseModel
from langgraph.graph import MessagesState, StateGraph, START, END
from typing import List
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langgraph.types import Send
import operator
from typing import Annotated
from typing import NamedTuple
from utils.fetch_docs import fetch_documents
from langchain_google_genai import ChatGoogleGenerativeAI





# Set up the logger
logging.basicConfig(
    level=logging.INFO,  # Set to DEBUG for detailed logs
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        # logging.FileHandler("scraper.log"),  # Log to a file
        logging.StreamHandler()  # Log to console
    ]
)

logger = logging.getLogger(__name__)


template = """Your job is to get information from a user about what kind of agent they wish to build.

You should get the following information from them:

- What the objective of the agent is
- Various usecases of the agent 
- Some examples of what the agent will be doing (Input and expected output pairs)

If you are not able to discern this info, ask them to clarify! Do not attempt to wildly guess.

After you are able to discern all the information, call the tool AgentInstruction"""

In [4]:
class AgentInstructions(BaseModel):
    """Instructions on how to build the Agent"""
    objective: str = Field(description= "What is the primary objective of the agent")
    usecases: List[str] = Field(description= "What are the various responsibilities of the agent which it needs to fulfill")
    examples : str = Field(description= "What are some examples of the usage of the agent (input query and expected output from the agent) ?")

class AgentBuilderState(MessagesState):
    agent_instructions: AgentInstructions = Field("the requirement analysis generated by the model.")
    json_code: str = Field("The json code generated")
    python_code: str = Field("The Python code generated")

class ArchitectureEvaluationState(MessagesState):
    agent_instructions: AgentInstructions = Field("the requirement analysis generated by the model.")
    url: str = Field("url of the agent architecture to evaluate against")

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-preview-05-20", temperature=0)

def requirement_analysis_node(state: AgentBuilderState):
    
    llm_with_tool = llm.bind_tools([AgentInstructions])
    response = llm_with_tool.invoke([SystemMessage(content=template)] + state["messages"])
    
      # Construct the final answer from the arguments of the last tool call  
    if len(response.tool_calls) == 0:
        return {"messages": [response]}
    
    agent_instructions = response.tool_calls[0]
    agent_instructions = AgentInstructions(**agent_instructions["args"])
    
    return {"messages": [response], "agent_instructions": agent_instructions}


def route_state(state: AgentBuilderState):
    messages = state["messages"]
    if isinstance(messages[-1], AIMessage) and messages[-1].tool_calls:
        return "agent_kernel_builder"
    elif not isinstance(messages[-1], HumanMessage):
        return END
    return "requirement_analysis"


In [5]:

AGENT_KERNEL_PROMPT = PromptTemplate.from_template(
    """
        Task Overview:
        Design a langgraph StateGraph object implementing the the best architecture for the given set of requirements, tailored to fulfill the user requirements defined below

        <Requirements>
        Objectives: {objective}
        usecases: {responsibilities}
        examples: {examples}
        </Requirements>

        
        Expected Output:
        Your task is to produce a compiled StateGraph object.

        Guidelines for Code Generation:
        - Accuracy: Avoid hallucinations or speculative assumptions when writing code. Refer exclusively to the provided documentation.
        - Understanding: Thoroughly comprehend the architecture and examples of code.
        - Customization: Generate code tailored specifically to meet the user requirements.

    """)

def agent_kernel_builder(state: AgentBuilderState):
    """Build the agent kernel using the best architecture."""
    agent_instructions : AgentInstructions = state["agent_instructions"]
    langgraph_glossary_url = "https://langchain-ai.github.io/langgraph/concepts/low_level/"
    # agent_architecture_report.name
    #agent_architecture_report.highlights
    #agent_architecture_report.justification
    #agent_architecture_report.tailored_design
    print("reached here")
    response =  llm.invoke([HumanMessage(content=AGENT_KERNEL_PROMPT.format(
        objective=agent_instructions.objective,
        responsibilities=agent_instructions.usecases,
        examples = agent_instructions.examples,
        # langgraph_glossary=fetch_documents(langgraph_glossary_url),
        ))])
    
    # Return the generated agent kernel as the output
    return {
        "messages": [AIMessage(content="Generated agent kernel code!")],
        "python_code": response.content,
    }

In [6]:

CODE_TO_JSON_PROMPT = PromptTemplate.from_template("""
You are tasked with converting the following stategraph comPilation code into a JSON. 

Contextual documents for understanding code:
{documents}

The Input code is as follows:
{code_snippet}

OUTPUT: Explaination and JSON. Do not include any code blocks. Seperate the JSON and explaination blocks and ensure that there is an explaination for each line of JSON produced but keep the blocks seperated.
Each Output JSON will have a nodes sections containing all the nodes and an edges section

Please follow:
1. Produce the explaination first and then the JSON after it. DO not produce the JSON first. 
2. For any conditional edges, please include all the nodes that the source of a conditional edge can reach as part of the explaination.
3. Any Edge entry in the JSON can only be conditional(mention conditional: true) if the source for that edge acts as a source for multiple edges. If you cannot point to atleast 2 targets for 1 source, then that source will not have any conditional edges
4. A source can have any number of targets. Please write the explaination for each source node to target node edge
5. Please ensure that the JSON starts with __START__ node and __END__ node with the correct edges from and to them
6. Ensure all elements in the nodes sections of the output json contain the following fields: Schema_info, input_schema, output_schema, description, function_name. Please do not return any entries in the nodes without these fields and these fields can't be empty
7. Ensure all elements in the edges sections of the output json contain the following fields: source, target, routing_conditions, conditional. Please do not return any entries in the edges without these fields and they can't be empty
8. Every node should be a part of atleast one edge, Please ensure this is followed
9. Attach the code snippet for each node aswell that. Please extract it from the Input code


Example output JSON for a node:
    "code_node":{{
        "schema_info": /"/"/"CodeWriterState:
      type: TypedDict
      fields:
      - name: user_query
        type: str
      - name: execution_result
        type: str/"/"/",
    "input_schema": "CodeWriterState",
    "output_schema":"RequiremenCodeWriterStatetAnalysisState",
    "description":"This node analyzes the user_query, if the query is to write a code, it will make a tool call to run the proposed code. This node returns command object",
    "function_name": "code_step"
    }}

Example output JSON for an edge:
edge:{{ source: "abc", target: "cde", routing_condition: "if abc made a tool call then go to cde", "conditional": true}}
edge:{{ source: "abc", target: "xyz", routing_condition: "if abc made an interupt to a human then go to xyz", "conditional": true}}
edge:{{ source: "xyz", target: "_END_", routing_condition: "no nodes to go after xyz, we have our final output for this path", "conditional": false}}s         
""")
def code_to_json_node(state: AgentBuilderState):
    """Convert the generated code to JSON."""
    langgraph_glossary_url = "https://langchain-ai.github.io/langgraph/concepts/low_level/"
    json_code_ouptut = llm.invoke([HumanMessage(content=CODE_TO_JSON_PROMPT.format(
        code_snippet=state["python_code"],
        documents = fetch_documents(langgraph_glossary_url),
        ))])
    
    # Return the JSON code as the output
    return {
        "messages": [AIMessage(content="Generated JSON code!")],
        "json_code": json_code_ouptut.content,
    }



In [7]:

JSON_CODE_COMBINE_PROMPT = PromptTemplate.from_template("""
You are tasked with verifying and updating the provided JSON that represents the nodes and edges of the langgraph to ensure it is correct with respect to the input code.

Contextual Documents for Understanding Code:
{documents}

Input Code:
<Input code>{code_snippet}</Input code>
JSON:
<JSON>{node_json}</JSON>

OUTPUT: JSON
Your task is divided into two parts:
- Validation: Verify that the JSON adheres to the rules outlined below.
- Correction and Augmentation: If the JSON is incorrect or incomplete, update it with a clear justification for every change, and include the code snippet for each node extracted from the input code.

Rules for Validation and Update:
- Conditional Edges:- An edge can be marked as conditional: true only if its source acts as a source for multiple edges (i.e., at least two targets).
- If the source does not meet this condition, then it cannot have conditional edges.

- Edge Targets:- Each source can have any number of targets. This flexibility must be maintained.

- Start and End Nodes:- The JSON must begin with the __START__ node and conclude with the __END__ node, with correct edges to and from them. Edges into the END node can also be conditional if they meet the above mentioned conditions      

- Node Structure:- Each node entry in the nodes section of the JSON must include the following non-empty fields:- schema_info
- id
- schema_info
- input_schema
- output_schema
- description
- function_name


- Edge Structure:- Each edge entry in the edges section of the JSON must include the following non-empty fields:- source
- target
- routing_conditions
- conditional


- Node-Edge Relationship:- Every node must be part of at least one edge. Ensure this relationship is consistently followed.

- Node Code Snippets:- Attach a code field to every node in the JSON, extracted directly from the input code.

- Schema Requirements:- The final JSON must conform to the schema provided below:


Schema for JSON. Ensure the following schema is followed. No field should be missing for any node or edge:
- Node Example:

{{
  "code_node": {{
    "id" : "<id of the node, similar to name>"
    "schema_info": "<define the class structure of the state of this node. Also provide a name>",
    "input_schema": "<input state object name>",
    "output_schema": "<output state object name>",
    "description": "<description>",
    "function_name": "<function_name>",
    "code": "<python_code>"
  }}
}}


- Edge Example:

{{
  "edge": {{
    "source": "<source_node>",
    "target": "<target_node>",
    "routing_condition": "<routing_condition>",
    "conditional": true/false
  }}
}}


                                            
Key Instructions:
- Do not update any pre-existing field of the JSON unless you have an extremely strong justification for doing so.
- Clearly document the reasoning behind any additions, updates, or modifications to the JSON. Justifications should draw inspiration from the contextual documents mentioned earlier.
- Ensure conditional edges strictly adhere to the rules outlined above.
- Input_Schema and output_schemas can only have value None in JSON for START and END nodes. Please follow this without fail
- Include a code field for each code_node entry, with the exact code that corresponds to the node in the input code.


""")

def json_better_node(state: AgentBuilderState):
    """Add code to the json flow"""
    langgraph_glossary_url = "https://langchain-ai.github.io/langgraph/concepts/low_level/"
    json_code_ouptut = llm.invoke([HumanMessage(content=JSON_CODE_COMBINE_PROMPT.format(
        code_snippet=state["python_code"],
        documents = fetch_documents(langgraph_glossary_url),
        node_json = state["json_code"]
        ))])
    
    # Return the JSON code as the output
    return {
        "messages": [AIMessage(content="Generated updated JSON code!")],
        "json_code": json_code_ouptut.content,
    }



In [11]:
CODE_GEN_PROMPT = PromptTemplate.from_template( """
You are an expert Python programmer specializing in LangGraph and AI agent development. Your primary task is to generate compilable Python code for a LangGraph state graph based on a JSON description that will be provided. You must carefully analyze the JSON to determine node functionalities, state management, and graph connectivity.

**Input:**
You will receive a JSON object with two main keys:
1.  `nodes`: A dictionary where each key is a unique node ID. The value for each node ID is an object containing:
    * `id`: The node's identifier.
    * `schema_info`: A string describing the structure of the `GraphState` (e.g., "GraphState:\n type: TypedDict\n fields:\n - name: input\n type: str..."). You will need to parse this to define the `GraphState` TypedDict.
    * `input_schema`: The expected input schema for the node (typically "GraphState").
    * `output_schema`: The schema of the output produced by the node (typically "GraphState", indicating a partial update).
    * `description`: A natural language description of what the node does. This is crucial for determining implementation strategy.
    * `function_name`: The suggested Python function name for this node.
    * `code` (optional): A string containing Python code for the node's function. This might be a complete implementation or a simplified example.

2.  `edges`: A list of objects, each describing a directed edge in the graph. Each edge object contains:
    * `source`: The ID of the source node (or "__START__" for the graph's entry point).
    * `target`: The ID of the target node (or "__END__" for a graph termination point).
    * `routing_conditions`: A natural language description of the condition under which this edge is taken, especially for conditional edges.
    * `conditional`: A boolean flag, `true` if the edge is part of a conditional branch, `false` otherwise.

**Output Requirements:**
Generate a single, self-contained, and compilable Python script that performs the following:

1.  **Imports:** Include all necessary Python libraries (e.g., `typing` for `TypedDict`, `Optional`, `Dict`, `Any`; `langgraph.graph.StatefulGraph`, `langgraph.graph.START`, `langgraph.graph.END`; `langgraph.checkpoint.memory.InMemoryCheckpointer`; potentially `langchain_openai`, `langchain_core.pydantic_v1` for Pydantic models if LLM with structured output is needed; `re` if used in provided code snippets).

2.  **State Definition (`GraphState`):**
    * Parse the `schema_info` string from one of the node descriptions (assume they are consistent; if not, use the first one encountered).
    * Define a `GraphState` class using `typing.TypedDict` that accurately reflects the fields and their types as specified in `schema_info`. Pay close attention to `Optional` types and nested structures if any.

3.  **Node Implementation (Python Functions):**
    For each node defined in the `nodes` section of the JSON:
    * Create a Python function with the `function_name` provided. This function must accept the `GraphState` (the TypedDict you defined) as its sole argument and return a dictionary representing the partial update to the state.
    * **Decision Logic for Implementation Strategy (CRITICAL):**
        * **Simple Algorithmic Logic:** If the `description` indicates a straightforward task and the provided `code` snippet is a complete, purely algorithmic Python function (e.g., string manipulation, regex-based parsing, simple dictionary operations) that fully implements this description without requiring advanced natural language understanding or complex inference, then directly use or adapt this provided `code` for the node's function.
        * **LLM Requirement:** If the `description` implies complex tasks such as:
            * Natural Language Understanding (NLU) (e.g., "robust intent classification", "understanding user queries")
            * Complex decision-making based on unstructured input
            * Text generation or summarization
            * Or if the `description` explicitly states "this function would typically use an LLM" or similar.
            Then, you MUST implement the node to utilize a Large Language Model (LLM).
            * If `code` is provided but is clearly a simplistic placeholder for what should be an LLM-driven task (e.g., basic keyword matching for intent classification when robust understanding is needed, as seen in the example's `classify_intent` node), you should replace or augment this simplistic code with an actual LLM integration.
            * When an LLM is required:
                * Include the necessary import for a common LLM client (e.g., `from langchain_openai import ChatOpenAI`).
                * In the node function, you can either instantiate a generic model (e.g., `llm = ChatOpenAI(model="gpt-3.5-turbo") # TODO: Replace with specific model and API key if needed`) or provide clear comment placeholders (e.g., `# Initialize LLM here (e.g., llm = ChatOpenAI(...))` followed by `# response = llm.invoke(...) # Adapt prompt and parsing based on node's task`).
                * The LLM should be prompted (even if schematically) to perform the task outlined in the node's `description`.
        * **Structured Output from LLM:** If the node's `description` or its purpose (e.g., extracting specific entities, parsing data into a predefined schema beyond simple `GraphState` fields) suggests that an LLM needs to produce a structured output (e.g., JSON, specific Pydantic model):
            * Define a Pydantic model (e.g., `from langchain_core.pydantic_v1 import BaseModel, Field`) representing the desired structured output.
            * If implementing an LLM call, configure it to use the Pydantic model for its output (e.g., with OpenAI's function calling/tool usage features, or by instructing the LLM to generate JSON conforming to the model).
        * **Tools for LLM:** If the `description` implies the node needs to interact with external systems, call specific APIs, or choose from a set of defined capabilities (e.g., "search customer database," "fetch external data," "invoke a specific service"):
            * Define appropriate LangChain tools (e.g., using `@tool` from `langchain_core.tools`).
            * If implementing an LLM call, bind these tools to the LLM. The LLM's role would be to decide which tool(s) to call based on the current state and input. For this generation task, you might define placeholder tools if the specifics are not in the JSON.

4.  **Graph Construction (`StatefulGraph`):**
    * Instantiate `StatefulGraph(GraphState)`.
    * Add each implemented node function to the graph using `graph.add_node("node_id", node_function)`.
    * Set the graph's entry point using `graph.add_edge(START, "entry_node_id")` where `"entry_node_id"` is the target of the edge originating from `"__START__"`.

5.  **Edge Implementation:**
    * Iterate through the `edges` list in the JSON.
    * **Regular Edges:** If `conditional` is `false`:
        * If `target` is `__END__`, use `graph.add_edge(source_node_id, END)`.
        * Otherwise, use `graph.add_edge(source_node_id, target_node_id)`.
    * **Conditional Edges:** If `conditional` is `true`:
        * The `source` node of these conditional edges is expected to produce some output in the `GraphState` (e.g., an `intent` field) that determines the next path.
        * Create a separate routing function (e.g., `def route_after_source_node(state: GraphState) -> str:`).
        * This routing function must inspect the relevant fields in the `state` and return the string ID of the next node to execute, based on the logic described in the `routing_conditions` for each conditional edge originating from that source.
        * Use `graph.add_conditional_edges(source_node_id, routing_function, {{ "target_id_1": "target_id_1", "target_id_2": "target_id_2", ... "__END__": END }})`. The keys in the dictionary are the possible return values from your routing function, and the values are the actual node IDs or `END`.

6.  **Compilation:**
    * Instantiate an `InMemoryCheckpointer`: `checkpointer = InMemoryCheckpointer()`.
    * Compile the graph: `final_app = graph.compile(checkpointer=checkpointer)`. The compiled graph must be assigned to a variable named `final_app`.

**Important Considerations:**
* The generated Python code must be complete, runnable, and accurately reflect the graph structure and logic described in the JSON.
* Ensure all node functions correctly update and return the relevant parts of the `GraphState`.
* If the provided `code` in the JSON uses specific libraries (e.g., `re`), make sure the corresponding import is included at the top of the script.
* Handle `__START__` and `__END__` correctly in edge definitions. `langgraph.graph.START` and `langgraph.graph.END` should be used.

Here is the JSON input:
<json>
{code_json}
</json>

Please generate the Python code now:
""")

def code_node(state: AgentBuilderState):
    """Produce the final python code"""
    code_ouptut = llm.invoke([HumanMessage(content=CODE_GEN_PROMPT.format(
        code_json = state["json_code"]
        ))])
    
    # Return the JSON code as the output
    return {
        "messages": [AIMessage(content="Generated final python code!")],
        "python_code": code_ouptut.content,
    }


In [23]:
CODE_GEN_PROMPT = PromptTemplate.from_template( """
You are an expert Python programmer specializing in LangGraph and AI agent development. Your primary task is to generate compilable, logical, and complete Python code for a LangGraph state graph based on a JSON description. You must prioritize LLM-based implementations for relevant tasks and consider advanced graph architectures.

**Input:**
You will receive a JSON object with two main keys:
1.  `nodes`: A dictionary where each key is a unique node ID. The value for each node ID is an object containing:
    * `id`: The node's identifier.
    * `schema_info`: A string describing the structure of the `GraphState` (e.g., "GraphState:\n type: TypedDict\n fields:\n - name: input\n type: str..."). You will need to parse this to define the `GraphState` TypedDict.
    * `input_schema`: The expected input schema for the node (typically "GraphState").
    * `output_schema`: The schema of the output produced by the node (typically "GraphState", indicating a partial update).
    * `description`: A natural language description of what the node does. This is crucial for determining implementation strategy and overall architecture.
    * `function_name`: The suggested Python function name for this node.
    * `code` (optional): A string containing Python code for the node's function. **Treat this `code` primarily as an illustration or a very basic version. Prioritize LLM-based solutions if the `description` suggests a more robust approach is needed.**

2.  `edges`: A list of objects, each describing a directed edge in the graph. Each edge object contains:
    * `source`: The ID of the source node (or "__START__" for the graph's entry point).
    * `target`: The ID of the target node (or "__END__" for a graph termination point).
    * `routing_conditions`: A natural language description of the condition under which this edge is taken, especially for conditional edges.
    * `conditional`: A boolean flag, `true` if the edge is part of a conditional branch, `false` otherwise.

---
**Phase 1: Graph Architecture Analysis & Strategy**

Before generating any code, analyze the overall workflow implied by the `nodes` and `edges` in the JSON.
1.  **Identify Potential Architectures:** Consider if the described system aligns with or would benefit from known advanced LangGraph architectures such as:
    * **Plan and Execute**: Does the graph imply a planning step (e.g., breaking down a complex task) followed by the execution of those plans by one or more action nodes?
    * **Agent Supervisor / Hierarchical Agent Teams**: Do the nodes and their conditional routing suggest a supervisory agent dispatching tasks to specialized worker agents, or a hierarchy of agents making decisions and delegating?
    * **Multi-Agent Collaboration (e.g., Swarm Architecture)**: Does the problem benefit from multiple agents working in parallel or collaboratively, perhaps sharing insights or contributing to a common goal?
    * **Reflection / Self-Correction (e.g., Self-Discover frameworks)**: Are there indications of iterative refinement, where results are evaluated and the process is adjusted?

2.  **Architectural Decision:**
    * If you determine that one or more of these architectures are strongly applicable and would create a more robust or intelligent system based on the JSON's intent, choose to implement it.
    * This may require you to define additional coordinating nodes or logic (e.g., a dedicated 'planner' LLM-agent, a 'supervisor' LLM-agent that routes tasks) that are not explicitly listed as individual nodes in the input JSON but are integral to the chosen architecture.
    * If no specific advanced architecture seems directly applicable or sufficiently justified by the input JSON, proceed with a standard stateful graph construction based on the explicit nodes and edges.

3.  **Initial Comment:** At the very beginning of your generated Python script, include a comment block stating:
    * Which LangGraph architecture(s) (if any) you've identified and chosen to implement, with a brief justification based on your interpretation of the input JSON.
    * If you are proceeding with a standard graph, mention that.

---
**Phase 2: Python Code Generation**

Generate a single, self-contained, and compilable Python script that implements your chosen strategy.

1.  **Imports:** Include all necessary Python libraries (e.g., `typing`, `langgraph.graph`, `langgraph.checkpoint.memory`, LLM client libraries like `langchain_openai`, `langchain_google_genai`, `langchain_core.pydantic_v1`, `langchain_core.tools`, `re`).

2.  **State Definition (`GraphState`):**
    * Parse the `schema_info` string to define a `GraphState` class using `typing.TypedDict`.

3.  **Node Implementation (Python Functions):**
    For each conceptual node in your chosen architecture (these may map directly to JSON nodes or be new architectural nodes you define):
    * Create a Python function. This function must accept the `GraphState` and return a dictionary representing the partial update to the state.
    * **Decision Logic for Implementation (Prioritize LLM, No Mock Data):**
        * **Default to LLM-Based Solutions:** Your default stance should be to implement an **LLM-based solution** if the node's `description` (from JSON or your architectural design) suggests tasks like:
            * Natural Language Understanding (NLU)
            * Complex classification or routing
            * Content generation or summarization
            * Tool selection and usage
            * Planning or complex decision-making.
            * Any task where an LLM would provide more robust, flexible, or intelligent behavior than simple hardcoded logic.
        * **Handling Provided `code`:** If `code` is present in the JSON for a node, treat it as a **low-priority hint or a simplistic example**. Do **not** simply copy it if an LLM approach is more appropriate for the described task.
        * **Algorithmic Logic (Use Sparingly):** Only use purely algorithmic Python code (like from the `code` attribute or written new) if the node's task is genuinely simple, deterministic (e.g., basic data formatting, fixed calculation), *and* an LLM would offer no significant benefit for that specific, narrow function.
        * **Functional LLM Calls:** When an LLM is used, instantiate a generic model (e.g., `llm = ChatOpenAI(model="gpt-3.5-turbo")` or `llm = ChatGoogleGenerativeAI(model="gemini-pro")`) and include a **functional, descriptive prompt** relevant to the node's task. Ensure the code for the LLM call is complete and not just a comment. Add a `TODO` comment for the user to specify API keys and potentially refine the model/prompt.
        * **No Mock Data:** Generated functions must be logical and aim for completeness. **Avoid using mock data or overly simplistic placeholder logic** where an LLM or a proper algorithmic implementation is expected.
        * **Structured Output & Tools:** If the task implies structured output from an LLM or the use of tools, define necessary Pydantic models and/or LangChain tools, and integrate them with the LLM call.
        * **Human in the Loop Nodes:** If you've designed a HITL step as a dedicated node, its function might primarily format data for human review and then process the subsequent human input (which would be added to the state, potentially by an external mechanism or a subsequent node). The graph might pause using an interruption mechanism tied to this node.
        * **Structured Output from LLM:** If the node's `description` or its purpose (e.g., extracting specific entities, parsing data into a predefined schema beyond simple `GraphState` fields) suggests that an LLM needs to produce a structured output (e.g., JSON, specific Pydantic model):
            * Define a Pydantic model (e.g., `from langchain_core.pydantic_v1 import BaseModel, Field`) representing the desired structured output.
            * If implementing an LLM call, configure it to use the Pydantic model for its output (e.g., with OpenAI's function calling/tool usage features, or by instructing the LLM to generate JSON conforming to the model).
        * **Tools for LLM:** If the `description` implies the node needs to interact with external systems, call specific APIs, or choose from a set of defined capabilities (e.g., "search customer database," "fetch external data," "invoke a specific service"):
            * Define appropriate LangChain tools (e.g., using `@tool` from `langchain_core.tools`).
            * If implementing an LLM call, bind these tools to the LLM. The LLM's role would be to decide which tool(s) to call based on the current state and input. For this generation task, you might define placeholder tools if the specifics are not in the JSON.
        * Ensure Variable assignment needs to be coherent with the graph state 

4.  **Graph Construction (`StatefulGraph`):**
    * Instantiate `StatefulGraph(GraphState)`.
    * Add each implemented node function to the graph using `graph.add_node("node_id", node_function)`.
    * Set the graph's entry point using `graph.add_edge(START, "entry_node_id")` where `"entry_node_id"` is the target of the edge originating from `"__START__"`.

5.  **Edge Implementation:**
    * Iterate through the `edges` list in the JSON.
    * **Regular Edges:** If `conditional` is `false`:
        * If `target` is `__END__`, use `graph.add_edge(source_node_id, END)`.
        * Otherwise, use `graph.add_edge(source_node_id, target_node_id)`.
    * **Conditional Edges:** If `conditional` is `true`:
        * The `source` node of these conditional edges is expected to produce some output in the `GraphState` (e.g., an `intent` field) that determines the next path.
        * Create a separate routing function (e.g., `def route_after_source_node(state: GraphState) -> str:`).
        * This routing function must inspect the relevant fields in the `state` and return the string ID of the next node to execute, based on the logic described in the `routing_conditions` for each conditional edge originating from that source.
        * Use `graph.add_conditional_edges(source_node_id, routing_function, {{ "target_id_1": "target_id_1", "target_id_2": "target_id_2", ... "__END__": END }})`. The keys in the dictionary are the possible return values from your routing function, and the values are the actual node IDs or `END`.

6.  **Compilation:**
    * Instantiate an `InMemoryCheckpointer`: `checkpointer = InMemoryCheckpointer()`.
    * Compile the graph: `final_app = graph.compile(checkpointer=checkpointer)`. The compiled graph must be assigned to a variable named `final_app`.

---
**Phase 3: Required Keys/Credentials Identification**

After generating the complete Python script, add a separate section at the end of your response, clearly titled:
`## Required Keys and Credentials`

In this section, list all environment variables or API keys a user would need to set for the generated code to execute successfully (e.g., `OPENAI_API_KEY`, `GOOGLE_API_KEY`, tool-specific keys). If no external keys are needed, state that.

---
**Important Considerations (General):**
* The primary goal is **compilable, logical, and functionally plausible Python code** that intelligently interprets the JSON input.
* Focus on creating a system that leverages LLMs effectively for tasks suited to them.
* Ensure node functions correctly update and return relevant parts of the `GraphState`.
* If the provided `code` in the JSON uses specific libraries (e.g., `re`), make sure the corresponding import is included at the top of the script.
* Handle `__START__` and `__END__` correctly in edge definitions. `langgraph.graph.START` and `langgraph.graph.END` should be used.

Here is the JSON input:
<json>
{code_json}
</json>

Please generate the Python code and the list of required keys now:
""")

def code_node(state: AgentBuilderState):
    """Produce the final python code"""
    code_ouptut = llm.invoke([HumanMessage(content=CODE_GEN_PROMPT.format(
        code_json = state["json_code"]
        ))])
    
    # Return the JSON code as the output
    return {
        "messages": [AIMessage(content="Generated final python code!")],
        "python_code": code_ouptut.content,
    }


In [24]:
workflow = StateGraph(AgentBuilderState)
workflow.add_node("requirement_analysis", requirement_analysis_node)
workflow.add_node("agent_kernel_builder", agent_kernel_builder)
workflow.add_node("code_to_json", code_to_json_node)
workflow.add_node("json_update", json_better_node)
workflow.add_node("code_node",code_node)

@workflow.add_node
def add_tool_message(state: AgentBuilderState):
    
   
    return {
        "messages": [
            ToolMessage(
                content="Requirements generated!",
                tool_call_id=state["messages"][-1].tool_calls[0]["id"],
            )
        ]
    }

workflow.add_edge("code_node", END)
workflow.add_edge("json_update", "code_node")
workflow.add_edge("code_to_json", "json_update")
workflow.add_edge("agent_kernel_builder", "code_to_json")
workflow.add_conditional_edges("requirement_analysis", route_state, ["agent_kernel_builder", "requirement_analysis", END])
workflow.add_edge(START, "requirement_analysis")
infograph = workflow.compile()

In [ ]:
import uuid

cached_human_responses = ["hi!", "rag prompt", "1 rag, 2 none, 3 no, 4 no", "red", "q"]
cached_response_index = 0
config = {"configurable": {"thread_id": str(uuid.uuid4())}}
while True:
    try:
        user = input("User (q/Q to quit): ")
    except:
        user = cached_human_responses[cached_response_index]
        cached_response_index += 1
    print(f"User (q/Q to quit): {user}")
    if user in {"q", "Q"}:
        print("AI: Byebye")
        break
    output = None
    for output in infograph.stream(
        {"messages": [HumanMessage(content=user)]}, config=config, stream_mode="updates"
    ):
        last_message = next(iter(output.values()))["messages"][-1]
        last_message.pretty_print()

    if output and "prompt" in output:
        print("Done!")
        

User (q/Q to quit): 1. Primary objective is handle customers, resolve their issues, fetch information for them and conflict to resolution 2. Use cases would be updating customer information, providing customers with the data they ask for, asking customers for their requirements and suggesting them products 3. example 1: input: what is the price of product x? output:  price of x is 300$ example 2: input: Please update my address to "delhi" output: Address updated, here are your new user details-> Name: abc, age 24, Address: Delhi Example 3: I am opening up a bakery and need to setup my supply chain and billing output: We have product x that will output your supply chain monitoring and product Y that would automate your billing and accoutning along with, I would also suggest product z which helps with customer out reach and is generally used by customers opening new business and fits your use case
================================== Ai Message ==================================
Tool Calls

In [ ]:
```json
{
  "nodes": {
    "classify_intent": {
      "id": "classify_intent",
      "schema_info": "GraphState:\n  type: TypedDict\n  fields:\n  - name: input\n    type: str\n  - name: output\n    type: str\n  - name: intent\n    type: Optional[str]\n  - name: customer_data\n    type: dict\n  - name: product_catalog\n    type: dict",
      "input_schema": "GraphState",
      "output_schema": "GraphState",
      "description": "Classifies the intent of the user's input. In a real-world scenario, this function would typically use an LLM (e.g., OpenAI, Anthropic) to perform robust intent classification. For this example, we use simple keyword matching for demonstration.",
      "function_name": "classify_intent",
      "code": "def classify_intent(state: GraphState) -> GraphState:\n    \"\"\"\n    Classifies the intent of the user's input.\n    In a real-world scenario, this function would typically use an LLM\n    (e.g., OpenAI, Anthropic) to perform robust intent classification.\n    For this example, we use simple keyword matching for demonstration.\n    \"\"\"\n    user_input = state[\"input\"].lower()\n    intent = None\n\n    if \"update\" in user_input or \"change my address\" in user_input:\n        intent = \"update_info\"\n    elif \"price of\" in user_input or \"what is\" in user_input or \"data\" in user_input or \"my address\" in user_input or \"my name\" in user_input:\n        intent = \"fetch_data\"\n    elif \"bakery\" in user_input or \"supply chain\" in user_input or \"billing\" in user_input or \"need to setup\" in user_input:\n        intent = \"product_suggestion\"\n    else:\n        intent = \"unclear\" # Fallback for unhandled or ambiguous cases\n\n    print(f\"---CLASSIFY INTENT---\": Input: '{state['input']}' -> Intent: '{intent}'\")\n    return {\"intent\": intent}"
    },
    "update_customer_info_agent": {
      "id": "update_customer_info_agent",
      "schema_info": "GraphState:\n  type: TypedDict\n  fields:\n  - name: input\n    type: str\n  - name: output\n    type: str\n  - name: intent\n    type: Optional[str]\n  - name: customer_data\n    type: dict\n  - name: product_catalog\n    type: dict",
      "input_schema": "GraphState",
      "output_schema": "GraphState",
      "description": "Handles updating customer information based on the parsed input. This function simulates interaction with a customer database.",
      "function_name": "update_customer_info_agent",
      "code": "def update_customer_info_agent(state: GraphState) -> GraphState:\n    \"\"\"\n    Handles updating customer information based on the parsed input.\n    This function simulates interaction with a customer database.\n    \"\"\"\n    user_input = state[\"input\"]\n    customer_data = state[\"customer_data\"].copy() # Create a mutable copy\n    output = \"Could not update information. Please specify what to update.\"\n\n    # Example: \"Please update my address to \"delhi\"\"\n    # Using regex to extract field and value\n    match = re.search(r\"update my (address|email|name|age) to \\\"?([a-zA-Z0-9\\s]+)\\\"\", user_input, re.IGNORECASE)\n    if match:\n        field = match.group(1).lower()\n        value = match.group(2).strip()\n        if field in customer_data:\n            customer_data[field] = value\n            # Construct output matching example format\n            output = (f\"Address updated, here are your new user details-> \"\n                      f\"Name: {customer_data.get('name')}, age {customer_data.get('age')}, \"\n                      f\"Address: {customer_data.get('address')}\")\n        else:\n            output = f\"Sorry, I cannot update the field '{field}'. Available fields are name, age, address, email.\"\n    else:\n        output = \"I understand you want to update information, but I couldn't parse the specific details. Could you please rephrase?\"\n\n    print(f\"---UPDATE CUSTOMER INFO AGENT---\": Output: '{output}'\")\n    return {\"customer_data\": customer_data, \"output\": output}"
    },
    "fetch_info_agent": {
      "id": "fetch_info_agent",
      "schema_info": "GraphState:\n  type: TypedDict\n  fields:\n  - name: input\n    type: str\n  - name: output\n    type: str\n  - name: intent\n    type: Optional[str]\n  - name: customer_data\n    type: dict\n  - name: product_catalog\n    type: dict",
      "input_schema": "GraphState",
      "output_schema": "GraphState",
      "description": "Handles fetching specific information for the customer from mock data.",
      "function_name": "fetch_info_agent",
      "code": "def fetch_info_agent(state: GraphState) -> GraphState:\n    \"\"\"\n    Handles fetching specific information for the customer from mock data.\n    \"\"\"\n    user_input = state[\"input\"].lower()\n    customer_data = state[\"customer_data\"]\n    output = \"I couldn't find the information you asked for.\"\n\n    if \"price of product x\" in user_input:\n        price = customer_data.get(\"product_x_price\")\n        if price:\n            output = f\"price of x is {price}$\"\n    elif \"price of product y\" in user_input:\n        price = customer_data.get(\"product_y_price\")\n        if price:\n            output = f\"price of y is {price}$\"\n    elif \"price of product z\" in user_input:\n        price = customer_data.get(\"product_z_price\")\n        if price:\n            output = f\"price of z is {price}$\"\n    elif \"my address\" in user_input:\n        address = customer_data.get(\"address\")\n        if address:\n            output = f\"Your current address is: {address}\"\n    elif \"my name\" in user_input:\n        name = customer_data.get(\"name\")\n        if name:\n            output = f\"Your name is: {name}\"\n    else:\n        output = \"I can fetch information about product prices or your personal details like address and name. What specific information are you looking for?\"\n\n    print(f\"---FETCH INFO AGENT---\": Output: '{output}'\")\n    return {\"output\": output}"
    },
    "product_suggestion_agent": {
      "id": "product_suggestion_agent",
      "schema_info": "GraphState:\n  type: TypedDict\n  fields:\n  - name: input\n    type: str\n  - name: output\n    type: str\n  - name: intent\n    type: Optional[str]\n  - name: customer_data\n    type: dict\n  - name: product_catalog\n    type: dict",
      "input_schema": "GraphState",
      "output_schema": "GraphState",
      "description": "Asks customers for their requirements and suggests products from the catalog. This function simulates a recommendation engine or a sales agent.",
      "function_name": "product_suggestion_agent",
      "code": "def product_suggestion_agent(state: GraphState) -> GraphState:\n    \"\"\"\n    Asks customers for their requirements and suggests products from the catalog.\n    This function simulates a recommendation engine or a sales agent.\n    \"\"\"\n    user_input = state[\"input\"].lower()\n    product_catalog = state[\"product_catalog\"]\n    output = \"I can help you find products. Please tell me more about your needs.\"\n\n    # Example: \"I am opening up a bakery and need to setup my supply chain and billing\"\n    suggested_products_info = []\n    if \"supply chain\" in user_input:\n        if \"product x\" in product_catalog:\n            suggested_products_info.append(product_catalog[\"product x\"])\n    if \"billing\" in user_input:\n        if \"product y\" in product_catalog:\n            suggested_products_info.append(product_catalog[\"product y\"])\n\n    # Always suggest product z as it fits \"new business\" and \"customer outreach\"\n    if \"product z\" in product_catalog and product_catalog[\"product z\"] not in suggested_products_info:\n        suggested_products_info.append(product_catalog[\"product z\"])\n\n    if suggested_products_info:\n        product_phrases = []\n        for p_info in suggested_products_info:\n            # Format: \"product x that will output your supply chain monitoring\"\n            product_phrases.append(f\"product {p_info['name'].lower().replace('product ', '')} that {p_info['description'].lower()}\")\n\n        # Construct the output string to match the example closely\n        if len(product_phrases) > 1:\n            # Separate the last product for \"along with, I would also suggest\"\n            last_product_phrase = product_phrases.pop()\n            output = \"We have \" + \", \".join(product_phrases)\n            output += f\" along with, I would also suggest {last_product_phrase} and fits your use case\"\n        else:\n            output = f\"We have {product_phrases[0]} and fits your use case\"\n\n        # Add the general \"used by customers opening new business\" part if product z was suggested\n        if any(p['name'] == 'Product Z' for p in suggested_products_info):\n            output = output.replace(\"and fits your use case\", \"and is generally used by customers opening new business and fits your use case\")\n\n    else:\n        output = \"Based on your requirements, I couldn't find specific products. Could you elaborate?\"\n\n    print(f\"---PRODUCT SUGGESTION AGENT---\": Output: '{output}'\")\n    return {\"output\": output}"
    }
  },
  "edges": [
    {
      "source": "__START__",
      "target": "classify_intent",
      "routing_conditions": "This is the entry point of the graph, routing initial user input to the intent classification node.",
      "conditional": false
    },
    {
      "source": "classify_intent",
      "target": "update_customer_info_agent",
      "routing_conditions": "If the classified intent is 'update_info'.",
      "conditional": true
    },
    {
      "source": "classify_intent",
      "target": "fetch_info_agent",
      "routing_conditions": "If the classified intent is 'fetch_data' or if the intent is 'unclear' (fallback).",
      "conditional": true
    },
    {
      "source": "classify_intent",
      "target": "product_suggestion_agent",
      "routing_conditions": "If the classified intent is 'product_suggestion'.",
      "conditional": true
    },
    {
      "source": "update_customer_info_agent",
      "target": "__END__",
      "routing_conditions": "The task of updating customer information is complete.",
      "conditional": false
    },
    {
      "source": "fetch_info_agent",
      "target": "__END__",
      "routing_conditions": "The task of fetching information is complete.",
      "conditional": false
    },
    {
      "source": "product_suggestion_agent",
      "target": "__END__",
      "routing_conditions": "The task of providing product suggestions is complete.",
      "conditional": false
    }
  ]
}
```

In [ ]:
To design the `langgraph` `StateGraph` object, we will implement an architecture that routes user queries to specialized "agents" based on the detected intent.

**Architecture Overview:**

1.  **State Definition (`GraphState`)**: A `TypedDict` to hold the current state of the conversation, including the user's input, the generated output, the classified intent, and mock customer/product data.
2.  **Nodes**:
    *   `classify_intent`: An initial node that analyzes the user's input to determine the primary intent (e.g., update info, fetch data, product suggestion). In a real application, this would typically involve an LLM call. For this example, it uses simple keyword matching.
    *   `update_customer_info_agent`: Handles requests to modify customer details.
    *   `fetch_info_agent`: Retrieves specific information for the customer.
    *   `product_suggestion_agent`: Recommends products based on user requirements.
3.  **Conditional Routing**: After `classify_intent`, a conditional edge will direct the flow to the appropriate agent node.
4.  **End Node**: All agent nodes will lead to the `END` of the graph, signifying the completion of the task.

**Detailed Implementation:**

```python
from typing import TypedDict, Optional
from langgraph.graph import StateGraph, END
import re # For parsing updates in the update_customer_info_agent

# 1. Define the Graph State
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        input: The user's initial query.
        output: The response generated by an agent.
        intent: The classified intent of the user's query (e.g., "update_info", "fetch_data", "product_suggestion").
        customer_data: A dictionary simulating customer information, mutable across nodes.
        product_catalog: A dictionary simulating available products, immutable for agents.
    """
    input: str
    output: str
    intent: Optional[str]
    customer_data: dict
    product_catalog: dict

# Mock data for demonstration purposes
MOCK_CUSTOMER_DATA = {
    "name": "John Doe",
    "age": 30,
    "address": "123 Main St, Anytown",
    "email": "john.doe@example.com",
    "product_x_price": 300,
    "product_y_price": 500,
    "product_z_price": 100
}

MOCK_PRODUCT_CATALOG = {
    "product x": {
        "name": "Product X",
        "description": "will output your supply chain monitoring",
        "price": 300
    },
    "product y": {
        "name": "Product Y",
        "description": "would automate your billing and accounting",
        "price": 500
    },
    "product z": {
        "name": "Product Z",
        "description": "helps with customer outreach and is generally used by customers opening new business",
        "price": 100
    }
}

# 2. Define the Nodes (Functions)

def classify_intent(state: GraphState) -> GraphState:
    """
    Classifies the intent of the user's input.
    In a real-world scenario, this function would typically use an LLM
    (e.g., OpenAI, Anthropic) to perform robust intent classification.
    For this example, we use simple keyword matching for demonstration.
    """
    user_input = state["input"].lower()
    intent = None

    if "update" in user_input or "change my address" in user_input:
        intent = "update_info"
    elif "price of" in user_input or "what is" in user_input or "data" in user_input or "my address" in user_input or "my name" in user_input:
        intent = "fetch_data"
    elif "bakery" in user_input or "supply chain" in user_input or "billing" in user_input or "need to setup" in user_input:
        intent = "product_suggestion"
    else:
        intent = "unclear" # Fallback for unhandled or ambiguous cases

    print(f"---CLASSIFY INTENT---: Input: '{state['input']}' -> Intent: '{intent}'")
    return {"intent": intent}

def update_customer_info_agent(state: GraphState) -> GraphState:
    """
    Handles updating customer information based on the parsed input.
    This function simulates interaction with a customer database.
    """
    user_input = state["input"]
    customer_data = state["customer_data"].copy() # Create a mutable copy
    output = "Could not update information. Please specify what to update."

    # Example: "Please update my address to "delhi""
    # Using regex to extract field and value
    match = re.search(r"update my (address|email|name|age) to \"?([a-zA-Z0-9\s]+)\"?", user_input, re.IGNORECASE)
    if match:
        field = match.group(1).lower()
        value = match.group(2).strip()
        if field in customer_data:
            customer_data[field] = value
            # Construct output matching example format
            output = (f"Address updated, here are your new user details-> "
                      f"Name: {customer_data.get('name')}, age {customer_data.get('age')}, "
                      f"Address: {customer_data.get('address')}")
        else:
            output = f"Sorry, I cannot update the field '{field}'. Available fields are name, age, address, email."
    else:
        output = "I understand you want to update information, but I couldn't parse the specific details. Could you please rephrase?"

    print(f"---UPDATE CUSTOMER INFO AGENT---: Output: '{output}'")
    return {"customer_data": customer_data, "output": output}

def fetch_info_agent(state: GraphState) -> GraphState:
    """
    Handles fetching specific information for the customer from mock data.
    """
    user_input = state["input"].lower()
    customer_data = state["customer_data"]
    output = "I couldn't find the information you asked for."

    if "price of product x" in user_input:
        price = customer_data.get("product_x_price")
        if price:
            output = f"price of x is {price}$"
    elif "price of product y" in user_input:
        price = customer_data.get("product_y_price")
        if price:
            output = f"price of y is {price}$"
    elif "price of product z" in user_input:
        price = customer_data.get("product_z_price")
        if price:
            output = f"price of z is {price}$"
    elif "my address" in user_input:
        address = customer_data.get("address")
        if address:
            output = f"Your current address is: {address}"
    elif "my name" in user_input:
        name = customer_data.get("name")
        if name:
            output = f"Your name is: {name}"
    else:
        output = "I can fetch information about product prices or your personal details like address and name. What specific information are you looking for?"

    print(f"---FETCH INFO AGENT---: Output: '{output}'")
    return {"output": output}

def product_suggestion_agent(state: GraphState) -> GraphState:
    """
    Asks customers for their requirements and suggests products from the catalog.
    This function simulates a recommendation engine or a sales agent.
    """
    user_input = state["input"].lower()
    product_catalog = state["product_catalog"]
    output = "I can help you find products. Please tell me more about your needs."

    # Example: "I am opening up a bakery and need to setup my supply chain and billing"
    suggested_products_info = []
    if "supply chain" in user_input:
        if "product x" in product_catalog:
            suggested_products_info.append(product_catalog["product x"])
    if "billing" in user_input:
        if "product y" in product_catalog:
            suggested_products_info.append(product_catalog["product y"])

    # Always suggest product z as it fits "new business" and "customer outreach"
    if "product z" in product_catalog and product_catalog["product z"] not in suggested_products_info:
        suggested_products_info.append(product_catalog["product z"])

    if suggested_products_info:
        product_phrases = []
        for p_info in suggested_products_info:
            # Format: "product x that will output your supply chain monitoring"
            product_phrases.append(f"product {p_info['name'].lower().replace('product ', '')} that {p_info['description'].lower()}")

        # Construct the output string to match the example closely
        if len(product_phrases) > 1:
            # Separate the last product for "along with, I would also suggest"
            last_product_phrase = product_phrases.pop()
            output = "We have " + ", ".join(product_phrases)
            output += f" along with, I would also suggest {last_product_phrase} and fits your use case"
        else:
            output = f"We have {product_phrases[0]} and fits your use case"

        # Add the general "used by customers opening new business" part if product z was suggested
        if any(p['name'] == 'Product Z' for p in suggested_products_info):
            output = output.replace("and fits your use case", "and is generally used by customers opening new business and fits your use case")

    else:
        output = "Based on your requirements, I couldn't find specific products. Could you elaborate?"

    print(f"---PRODUCT SUGGESTION AGENT---: Output: '{output}'")
    return {"output": output}

# 3. Define the Conditional Edge Logic
def route_to_agent(state: GraphState) -> str:
    """
    Routes the execution based on the classified intent.
    """
    intent = state["intent"]
    if intent == "update_info":
        return "update_customer_info_agent"
    elif intent == "fetch_data":
        return "fetch_info_agent"
    elif intent == "product_suggestion":
        return "product_suggestion_agent"
    else:
        # Fallback for unclear intent: could route to a "clarify_agent"
        # For this example, we'll route to fetch_info_agent as a general fallback.
        print("---ROUTER---: Intent unclear, routing to fetch_info_agent as a default fallback.")
        return "fetch_info_agent"

# 4. Build the StateGraph
def create_customer_service_graph() -> StateGraph:
    """
    Creates and compiles the StateGraph for customer service operations.
    """
    workflow = StateGraph(GraphState)

    # Add nodes to the graph
    workflow.add_node("classify_intent", classify_intent)
    workflow.add_node("update_customer_info_agent", update_customer_info_agent)
    workflow.add_node("fetch_info_agent", fetch_info_agent)
    workflow.add_node("product_suggestion_agent", product_suggestion_agent)

    # Set the entry point for the graph
    workflow.set_entry_point("classify_intent")

    # Add conditional edges from the classify_intent node
    workflow.add_conditional_edges(
        "classify_intent",
        route_to_agent,
        {
            "update_customer_info_agent": "update_customer_info_agent",
            "fetch_info_agent": "fetch_info_agent",
            "product_suggestion_agent": "product_suggestion_agent",
            # If you had an "unclear_intent_agent", you'd add it here:
            # "unclear_intent_agent": "unclear_intent_agent"
        }
    )

    # Add direct edges from each agent node to the END node
    workflow.add_edge("update_customer_info_agent", END)
    workflow.add_edge("fetch_info_agent", END)
    workflow.add_edge("product_suggestion_agent", END)

    # Compile the graph
    app = workflow.compile()
    return app

# Compile the StateGraph object
compiled_customer_service_graph = create_customer_service_graph()

# Example Usage (for testing the compiled graph)
if __name__ == "__main__":
    # Initial state for the graph execution
    initial_state = {
        "input": "",
        "output": "",
        "intent": None,
        "customer_data": MOCK_CUSTOMER_DATA.copy(), # Use a copy to avoid modifying global mock data
        "product_catalog": MOCK_PRODUCT_CATALOG
    }

    print("--- Running Example 1: Fetch Information ---")
    ex1_state = initial_state.copy()
    ex1_state["input"] = "what is the price of product x?"
    result1 = compiled_customer_service_graph.invoke(ex1_state)
    print(f"Final Output 1: {result1['output']}\n")
    # Expected: price of x is 300$

    print("--- Running Example 2: Update Customer Information ---")
    ex2_state = initial_state.copy()
    ex2_state["input"] = 'Please update my address to "delhi"'
    result2 = compiled_customer_service_graph.invoke(ex2_state)
    print(f"Final Output 2: {result2['output']}")
    print(f"Updated Customer Data 2: {result2['customer_data']}\n")
    # Expected: Address updated, here are your new user details-> Name: John Doe, age 30, Address: delhi

    print("--- Running Example 3: Product Suggestion ---")
    ex3_state = initial_state.copy()
    ex3_state["input"] = "I am opening up a bakery and need to setup my supply chain and billing"
    result3 = compiled_customer_service_graph.invoke(ex3_state)
    print(f"Final Output 3: {result3['output']}\n")
    # Expected: We have product x that will output your supply chain monitoring, product y that would automate your billing and accounting along with, I would also suggest product z which helps with customer outreach and is generally used by customers opening new business and fits your use case

    print("--- Running Example 4: Fetch Customer Address ---")
    ex4_state = initial_state.copy()
    ex4_state["input"] = "What is my current address?"
    result4 = compiled_customer_service_graph.invoke(ex4_state)
    print(f"Final Output 4: {result4['output']}\n")

    print("--- Running Example 5: Unclear Intent (Fallback) ---")
    ex5_state = initial_state.copy()
    ex5_state["input"] = "Tell me a joke."
    result5 = compiled_customer_service_graph.invoke(ex5_state)
    print(f"Final Output 5: {result5['output']}\n")
```

In [ ]:
```python
import re
from typing import TypedDict, Optional, Dict, Any

from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemoryCheckpointer
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Define the GraphState
# Parsed from schema_info:
# GraphState:
#   type: TypedDict
#   fields:
#   - name: input
#     type: str
#   - name: output
#     type: str
#   - name: intent
#     type: Optional[str]
#   - name: customer_data
#     type: dict
#   - name: product_catalog
#     type: dict
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        input: The user's initial input string.
        output: The response generated by the agent.
        intent: The classified intent of the user's input (e.g., "update_info", "fetch_data").
        customer_data: A dictionary holding mock customer information.
        product_catalog: A dictionary holding mock product information.
    """
    input: str
    output: str
    intent: Optional[str]
    customer_data: Dict[str, Any]
    product_catalog: Dict[str, Any]

# --- Node Implementations ---

def classify_intent(state: GraphState) -> GraphState:
    """
    Classifies the intent of the user's input using an LLM.
    This replaces the simple keyword matching with a more robust LLM-based classification.
    """
    user_input = state["input"]
    
    # Initialize LLM
    # TODO: Replace with specific model and API key if needed
    # Ensure OPENAI_API_KEY environment variable is set or pass it directly.
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0) 

    # Define the prompt for intent classification
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are an expert intent classifier. Classify the user's input into one of the following categories:\n"
                "- 'update_info': If the user wants to change or update their personal information (e.g., address, name, email).\n"
                "- 'fetch_data': If the user wants to retrieve specific information (e.g., product prices, their address, their name).\n"
                "- 'product_suggestion': If the user is asking for product recommendations or solutions for a business need.\n"
                "- 'unclear': If the intent is ambiguous or does not fit the above categories.\n"
                "Respond with only the intent category string (e.g., 'update_info', 'fetch_data', 'product_suggestion', 'unclear')."
            ),
            ("human", "{input}"),
        ]
    )

    # Create a chain to classify intent
    intent_classifier_chain = prompt | llm | StrOutputParser()

    try:
        # Invoke the LLM to get the intent
        raw_intent = intent_classifier_chain.invoke({"input": user_input}).strip().lower()
        
        # Validate and map the LLM's output to expected intents
        valid_intents = ["update_info", "fetch_data", "product_suggestion", "unclear"]
        if raw_intent in valid_intents:
            intent = raw_intent
        else:
            intent = "unclear" # Fallback if LLM returns an unexpected string
            print(f"Warning: LLM returned unexpected intent '{raw_intent}'. Falling back to 'unclear'.")

    except Exception as e:
        print(f"Error during LLM intent classification: {e}. Falling back to 'unclear'.")
        intent = "unclear" # Fallback in case of API errors

    print(f"---CLASSIFY INTENT---: Input: '{state['input']}' -> Intent: '{intent}'")
    return {"intent": intent}

def update_customer_info_agent(state: GraphState) -> GraphState:
    """
    Handles updating customer information based on the parsed input.
    This function simulates interaction with a customer database.
    """
    user_input = state["input"]
    customer_data = state["customer_data"].copy() # Create a mutable copy
    output = "Could not update information. Please specify what to update."

    # Example: "Please update my address to "delhi""
    # Using regex to extract field and value
    match = re.search(r"update my (address|email|name|age) to \"?([a-zA-Z0-9\s]+)\"?", user_input, re.IGNORECASE)
    if match:
        field = match.group(1).lower()
        value = match.group(2).strip()
        if field in customer_data:
            customer_data[field] = value
            # Construct output matching example format
            output = (f"Address updated, here are your new user details-> "
                      f"Name: {customer_data.get('name')}, age {customer_data.get('age')}, "
                      f"Address: {customer_data.get('address')}")
        else:
            output = f"Sorry, I cannot update the field '{field}'. Available fields are name, age, address, email."
    else:
        output = "I understand you want to update information, but I couldn't parse the specific details. Could you please rephrase?"

    print(f"---UPDATE CUSTOMER INFO AGENT---: Output: '{output}'")
    return {"customer_data": customer_data, "output": output}

def fetch_info_agent(state: GraphState) -> GraphState:
    """
    Handles fetching specific information for the customer from mock data.
    """
    user_input = state["input"].lower()
    customer_data = state["customer_data"]
    output = "I couldn't find the information you asked for."

    if "price of product x" in user_input:
        price = customer_data.get("product_x_price")
        if price:
            output = f"price of x is {price}$"
    elif "price of product y" in user_input:
        price = customer_data.get("product_y_price")
        if price:
            output = f"price of y is {price}$"
    elif "price of product z" in user_input:
        price = customer_data.get("product_z_price")
        if price:
            output = f"price of z is {price}$"
    elif "my address" in user_input:
        address = customer_data.get("address")
        if address:
            output = f"Your current address is: {address}"
    elif "my name" in user_input:
        name = customer_data.get("name")
        if name:
            output = f"Your name is: {name}"
    else:
        output = "I can fetch information about product prices or your personal details like address and name. What specific information are you looking for?"

    print(f"---FETCH INFO AGENT---: Output: '{output}'")
    return {"output": output}

def product_suggestion_agent(state: GraphState) -> GraphState:
    """
    Asks customers for their requirements and suggests products from the catalog.
    This function simulates a recommendation engine or a sales agent.
    """
    user_input = state["input"].lower()
    product_catalog = state["product_catalog"]
    output = "I can help you find products. Please tell me more about your needs."

    # Example: "I am opening up a bakery and need to setup my supply chain and billing"
    suggested_products_info = []
    if "supply chain" in user_input:
        if "product x" in product_catalog:
            suggested_products_info.append(product_catalog["product x"])
    if "billing" in user_input:
        if "product y" in product_catalog:
            suggested_products_info.append(product_catalog["product y"])

    # Always suggest product z as it fits "new business" and "customer outreach"
    if "product z" in product_catalog and product_catalog["product z"] not in suggested_products_info:
        suggested_products_info.append(product_catalog["product z"])

    if suggested_products_info:
        product_phrases = []
        for p_info in suggested_products_info:
            # Format: "product x that will output your supply chain monitoring"
            product_phrases.append(f"product {p_info['name'].lower().replace('product ', '')} that {p_info['description'].lower()}")

        # Construct the output string to match the example closely
        if len(product_phrases) > 1:
            # Separate the last product for "along with, I would also suggest"
            last_product_phrase = product_phrases.pop()
            output = "We have " + ", ".join(product_phrases)
            output += f" along with, I would also suggest {last_product_phrase} and fits your use case"
        else:
            output = f"We have {product_phrases[0]} and fits your use case"

        # Add the general "used by customers opening new business" part if product z was suggested
        if any(p['name'] == 'Product Z' for p in suggested_products_info):
            output = output.replace("and fits your use case", "and is generally used by customers opening new business and fits your use case")

    else:
        output = "Based on your requirements, I couldn't find specific products. Could you elaborate?"

    print(f"---PRODUCT SUGGESTION AGENT---: Output: '{output}'")
    return {"output": output}

# --- Routing Function ---

def route_intent(state: GraphState) -> str:
    """
    Routes the graph based on the classified intent.
    """
    intent = state.get("intent")
    if intent == "update_info":
        print("---ROUTING---: Intent 'update_info' -> update_customer_info_agent")
        return "update_customer_info_agent"
    elif intent == "fetch_data":
        print("---ROUTING---: Intent 'fetch_data' -> fetch_info_agent")
        return "fetch_info_agent"
    elif intent == "product_suggestion":
        print("---ROUTING---: Intent 'product_suggestion' -> product_suggestion_agent")
        return "product_suggestion_agent"
    else: # Handles "unclear" and any other unexpected intent
        print("---ROUTING---: Intent 'unclear' or unhandled -> fetch_info_agent (fallback)")
        return "fetch_info_agent" # Fallback to fetch_info_agent for unclear cases

# --- Graph Construction ---

# Define the graph
graph = StateGraph(GraphState)

# Add nodes
graph.add_node("classify_intent", classify_intent)
graph.add_node("update_customer_info_agent", update_customer_info_agent)
graph.add_node("fetch_info_agent", fetch_info_agent)
graph.add_node("product_suggestion_agent", product_suggestion_agent)

# Set entry point
graph.add_edge(START, "classify_intent")

# Add conditional edges from classify_intent
graph.add_conditional_edges(
    "classify_intent",
    route_intent,
    {
        "update_customer_info_agent": "update_customer_info_agent",
        "fetch_info_agent": "fetch_info_agent",
        "product_suggestion_agent": "product_suggestion_agent",
        # The routing function handles "unclear" by returning "fetch_info_agent"
    },
)

# Add direct edges to END
graph.add_edge("update_customer_info_agent", END)
graph.add_edge("fetch_info_agent", END)
graph.add_edge("product_suggestion_agent", END)

# Compile the graph
checkpointer = InMemoryCheckpointer()
final_app = graph.compile(checkpointer=checkpointer)

# --- Example Usage (Optional, for testing) ---
if __name__ == "__main__":
    # Mock initial data for customer and product catalog
    initial_customer_data = {
        "name": "John Doe",
        "age": 30,
        "address": "123 Main St, Anytown",
        "email": "john.doe@example.com",
        "product_x_price": 100,
        "product_y_price": 250,
        "product_z_price": 500
    }

    initial_product_catalog = {
        "product x": {
            "name": "Product X",
            "description": "will output your supply chain monitoring"
        },
        "product y": {
            "name": "Product Y",
            "description": "will help you with your billing and invoicing"
        },
        "product z": {
            "name": "Product Z",
            "description": "is designed for customer outreach and new business setup"
        }
    }

    print("\n--- Running example 1: Update Info ---")
    inputs_1 = {
        "input": "Please update my address to \"456 Oak Ave, New City\"",
        "customer_data": initial_customer_data,
        "product_catalog": initial_product_catalog
    }
    for s in final_app.stream(inputs_1):
        print(s)
    print(f"Final State 1: {final_app.get_state(None).values['output']}")
    print(f"Updated Customer Data 1: {final_app.get_state(None).values['customer_data']}")


    print("\n--- Running example 2: Fetch Info ---")
    inputs_2 = {
        "input": "What is the price of product y?",
        "customer_data": initial_customer_data,
        "product_catalog": initial_product_catalog
    }
    for s in final_app.stream(inputs_2):
        print(s)
    print(f"Final State 2: {final_app.get_state(None).values['output']}")

    print("\n--- Running example 3: Product Suggestion ---")
    inputs_3 = {
        "input": "I am opening up a bakery and need to setup my supply chain and billing",
        "customer_data": initial_customer_data,
        "product_catalog": initial_product_catalog
    }
    for s in final_app.stream(inputs_3):
        print(s)
    print(f"Final State 3: {final_app.get_state(None).values['output']}")

    print("\n--- Running example 4: Unclear Intent (should fallback to fetch_info_agent) ---")
    inputs_4 = {
        "input": "Tell me about the weather.",
        "customer_data": initial_customer_data,
        "product_catalog": initial_product_catalog
    }
    for s in final_app.stream(inputs_4):
        print(s)
    print(f"Final State 4: {final_app.get_state(None).values['output']}")

    print("\n--- Running example 5: Fetch My Address ---")
    inputs_5 = {
        "input": "Can you tell me my address?",
        "customer_data": initial_customer_data,
        "product_catalog": initial_product_catalog
    }
    for s in final_app.stream(inputs_5):
        print(s)
    print(f"Final State 5: {final_app.get_state(None).values['output']}")
```

In [ ]:
```python
import json
import operator
from typing import Annotated, Any, Dict, List, Literal, Optional, TypedDict

from langchain_core.messages import BaseMessage
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import InMemoryCheckpointer
from langgraph.graph import END, START, StateGraph

# TODO: Set your OpenAI API key as an environment variable: OPENAI_API_KEY
# For example: os.environ["OPENAI_API_KEY"] = "your_api_key_here"
# Or pass it directly to ChatOpenAI(api_key="...")
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0) # Using a smaller, faster model for routing and extraction

# --- Phase 1: Graph Architecture Analysis & Strategy ---
# This graph implements a "Router-Agent" or "Agent Supervisor" architecture.
# Justification:
# 1.  **Intent Routing**: The `intent_router` node acts as a central supervisor,
#     classifying the user's initial input and dynamically routing it to
#     specialized "worker" nodes (`log_diet`, `log_workout`, `analyze_workout`).
#     This is a classic pattern for dispatching tasks based on user intent.
# 2.  **Specialized Agents**: Each action node (`log_diet`, `log_workout`, `analyze_workout`)
#     is designed to handle a specific type of task, leveraging LLMs for
#     structured data extraction and intelligent analysis. This modularity
#     is characteristic of agentic workflows.
# 3.  **Unified Response Generation**: A final `generate_response` node consolidates
#     outputs from various action paths, ensuring a consistent user experience.
# This architecture allows for clear separation of concerns and easy extension
# with new capabilities.

# --- Phase 2: Python Code Generation ---

# 1. State Definition
class FoodItem(TypedDict):
    name: str
    calories: int
    meal_type: str  # e.g., breakfast, lunch, dinner, snack, drink

class WorkoutExercise(TypedDict):
    name: str
    sets: int
    reps: int
    calories_burnt: int
->Variable assignment needs to be coherent with the graph state 
class GraphState(TypedDict):
    user_input: str
    parsed_intent: Optional[str]  # 'log_diet', 'log_workout', 'analyze_workout', 'unclear'
    diet_log: List[FoodItem]
    workout_log: List[WorkoutExercise]
    daily_calories_consumed: int
    daily_calories_burnt: int
    weekly_workout_summary: Dict[str, Any]  # e.g., {'bicep_curls_total_sets': 10, 'bicep_goal_achieved': False}
    analysis_result: Optional[str]
    response: Optional[str]

# Pydantic models for structured output from LLMs
class FoodItemPydantic(BaseModel):
    name: str = Field(description="Name of the food item.")
    calories: int = Field(description="Estimated calorie content of the food item.")
    meal_type: str = Field(description="Type of meal (e.g., breakfast, lunch, dinner, snack, drink, unknown).")

class WorkoutExercisePydantic(BaseModel):
    name: str = Field(description="Name of the exercise.")
    sets: int = Field(description="Number of sets performed.")
    reps: int = Field(description="Number of repetitions per set.")
    calories_burnt: int = Field(description="Estimated calories burnt during this exercise.")


# 2. Node Implementations (Python Functions)

def intent_router(state: GraphState) -> GraphState:
    """
    Classifies the user's initial input to determine the primary intent:
    'log_diet', 'log_workout', 'analyze_workout', or 'unclear'.
    """
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are an AI assistant that classifies user intent for a diet and workout tracker. "
                   "Classify the following user input into one of these categories: "
                   "'log_diet', 'log_workout', 'analyze_workout', 'unclear'. "
                   "Respond with only the category name, e.g., 'log_diet'."),
        ("user", "{user_input}")
    ])
    chain = prompt | llm | StrOutputParser()
    
    user_input = state.get("user_input", "")
    if not user_input:
        return {"parsed_intent": "unclear"}

    intent = chain.invoke({"user_input": user_input}).strip().lower()
    
    # Ensure the intent is one of the expected values
    valid_intents = ['log_diet', 'log_workout', 'analyze_workout', 'unclear']
    if intent not in valid_intents:
        intent = 'unclear' # Default to unclear if LLM hallucinates

    print(f"Intent Router: User input '{user_input}' classified as '{intent}'")
    return {"parsed_intent": intent}

def log_diet(state: GraphState) -> GraphState:
    """
    Processes user input related to diet. Extracts food items, estimates their
    calorie content, and updates the `diet_log` and `daily_calories_consumed` fields.
    Constructs a user-friendly response message.
    """
    parser = JsonOutputParser(pydantic_object=List[FoodItemPydantic])
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are an AI assistant that extracts food items and their details from user input. "
                   "Extract 'name', 'calories' (integer, estimate if not provided), and 'meal_type' "
                   "(e.g., 'breakfast', 'lunch', 'dinner', 'snack', 'drink', 'unknown'). "
                   "If no specific meal type is mentioned, use 'unknown'. "
                   "If no calories are mentioned, provide a reasonable estimate (e.g., apple: 95, chicken breast: 200, soda: 150). "
                   "Provide a JSON array of objects. If no food is mentioned, return an empty array. "
                   "Example: [{{\"name\": \"apple\", \"calories\": 95, \"meal_type\": \"snack\"}}].\n"
                   "Format instructions: {format_instructions}"),
        ("user", "{user_input}")
    ]).partial(format_instructions=parser.get_format_instructions())
    
    chain = prompt | llm.with_structured_output(List[FoodItemPydantic])

    user_input = state.get("user_input", "")
    extracted_food_items_pydantic: List[FoodItemPydantic] = []
    try:
        extracted_food_items_pydantic = chain.invoke({"user_input": user_input})
    except Exception as e:
        print(f"Error extracting food items: {e}")
        # Fallback to empty list if extraction fails

    current_diet_log: List[FoodItem] = state.get("diet_log", [])
    current_calories_consumed: int = state.get("daily_calories_consumed", 0)

    logged_items_names = []
    for item_pydantic in extracted_food_items_pydantic:
        item_typeddict: FoodItem = {
            "name": item_pydantic.name,
            "calories": item_pydantic.calories,
            "meal_type": item_pydantic.meal_type
        }
        current_diet_log.append(item_typeddict)
        current_calories_consumed += item_typeddict.get("calories", 0)
        logged_items_names.append(item_typeddict.get("name", "unknown item"))

    if logged_items_names:
        response_msg = f"Logged {', '.join(logged_items_names)} as part of your diet."
    else:
        response_msg = "Could not identify any food items to log."
    
    response_msg += f" Total calories consumed today: {current_calories_consumed}."

    print(f"Log Diet: {response_msg}")
    return {
        "diet_log": current_diet_log,
        "daily_calories_consumed": current_calories_consumed,
        "response": response_msg
    }

def log_workout(state: GraphState) -> GraphState:
    """
    Processes user input related to workouts. Extracts exercise details (sets, reps,
    estimated calories burnt), updates the `workout_log`, `daily_calories_burnt`,
    and `weekly_workout_summary` fields. Checks for and reports on workout goals
    and generates a response message.
    """
    parser = JsonOutputParser(pydantic_object=List[WorkoutExercisePydantic])
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are an AI assistant that extracts workout exercises and their details from user input. "
                   "Extract 'name', 'sets' (integer), 'reps' (integer), and 'calories_burnt' (integer, estimate if not provided). "
                   "If no specific reps are mentioned, use 10. If no calories are mentioned, estimate based on typical exercise intensity "
                   "(e.g., 20-50 calories per set, or 100-300 for a full workout). "
                   "Provide a JSON array of objects. If no workout is mentioned, return an empty array. "
                   "Example: [{{\"name\": \"bicep curls\", \"sets\": 4, \"reps\": 10, \"calories_burnt\": 80}}].\n"
                   "Format instructions: {format_instructions}"),
        ("user", "{user_input}")
    ]).partial(format_instructions=parser.get_format_instructions())
    
    chain = prompt | llm.with_structured_output(List[WorkoutExercisePydantic])

    user_input = state.get("user_input", "")
    extracted_exercises_pydantic: List[WorkoutExercisePydantic] = []
    try:
        extracted_exercises_pydantic = chain.invoke({"user_input": user_input})
    except Exception as e:
        print(f"Error extracting workout exercises: {e}")
        # Fallback to empty list if extraction fails

    current_workout_log: List[WorkoutExercise] = state.get("workout_log", [])
    current_calories_burnt: int = state.get("daily_calories_burnt", 0)
    current_weekly_summary: Dict[str, Any] = state.get("weekly_workout_summary", {})

    logged_exercises_names = []
    for exercise_pydantic in extracted_exercises_pydantic:
        exercise_typeddict: WorkoutExercise = {
            "name": exercise_pydantic.name,
            "sets": exercise_pydantic.sets,
            "reps": exercise_pydantic.reps,
            "calories_burnt": exercise_pydantic.calories_burnt
        }
        current_workout_log.append(exercise_typeddict)
        current_calories_burnt += exercise_typeddict.get("calories_burnt", 0)
        logged_exercises_names.append(exercise_typeddict.get("name", "unknown exercise"))
        
        # Update weekly summary for analysis (example: bicep curls goal)
        exercise_name_key = exercise_typeddict['name'].lower().replace(' ', '_') + '_total_sets'
        current_weekly_summary[exercise_name_key] = current_weekly_summary.get(exercise_name_key, 0) + exercise_typeddict['sets']
        
    # Simple goal check for bicep training (example: 7 sets per week)
    if current_weekly_summary.get('bicep_curls_total_sets', 0) >= 7:
        current_weekly_summary['bicep_goal_achieved'] = True
    else:
        current_weekly_summary['bicep_goal_achieved'] = False

    if logged_exercises_names:
        response_msg = f"Workout logged, {current_calories_burnt} calories burnt during the workout today."
    else:
        response_msg = "Could not identify any workouts to log."

    if current_weekly_summary.get('bicep_goal_achieved'):
        response_msg += " You have achieved the weekly goal for bicep training."

    print(f"Log Workout: {response_msg}")
    return {
        "workout_log": current_workout_log,
        "daily_calories_burnt": current_calories_burnt,
        "weekly_workout_summary": current_weekly_summary,
        "response": response_msg
    }

def analyze_workout(state: GraphState) -> GraphState:
    """
    Provides insights and recommendations based on the aggregated `weekly_workout_summary`.
    Generates an analysis output, which also serves as the response message.
    """
    weekly_summary = state.get("weekly_workout_summary", {})
    workout_log = state.get("workout_log", [])

    if not weekly_summary and not workout_log:
        analysis_output = "No workout data available for analysis yet. Log some workouts first!"
    else:
        prompt = ChatPromptTemplate.from_messages([
            ("system", "You are an AI fitness coach. Analyze the provided workout summary and log. "
                       "Provide insights, identify trends, suggest improvements, and motivate the user. "
                       "Consider total sets, reps, calories burnt, and any specific goals mentioned (like bicep goal). "
                       "If the bicep goal is achieved, congratulate the user and suggest next steps. "
                       "If not, encourage them to reach it. Keep the analysis concise and actionable."),
            ("user", "Here is the weekly workout summary: {weekly_summary}\n"
                     "Here is the full workout log: {workout_log}\n"
                     "Please provide an analysis and recommendations.")
        ])
        chain = prompt | llm | StrOutputParser()
        
        analysis_output = chain.invoke({
            "weekly_summary": json.dumps(weekly_summary, indent=2),
            "workout_log": json.dumps(workout_log, indent=2)
        })

    print(f"Analyze Workout: {analysis_output}")
    return {
        "analysis_result": analysis_output,
        "response": analysis_output
    }

def generate_response(state: GraphState) -> GraphState:
    """
    Formulates the final user-facing message. It prioritizes any response
    already set by a preceding action node. If no specific response is available
    (e.g., for an unclear intent), it provides a default helpful message.
    """
    if state.get("response"):
        final_response = state["response"]
    else:
        final_response = "I'm not sure how to help with that. Please tell me if you want to log diet, log a workout, or get a workout analysis."
    
    print(f"Generate Response: {final_response}")
    return {"response": final_response}

# 3. Graph Construction
graph_builder = StateGraph(GraphState)

# Add nodes
graph_builder.add_node("intent_router", intent_router)
graph_builder.add_node("log_diet", log_diet)
graph_builder.add_node("log_workout", log_workout)
graph_builder.add_node("analyze_workout", analyze_workout)
graph_builder.add_node("generate_response", generate_response)

# Set entry point
graph_builder.add_edge(START, "intent_router")

# Conditional routing from intent_router
def route_intent(state: GraphState) -> str:
    """Routes based on the parsed intent."""
    intent = state.get("parsed_intent")
    if intent == "log_diet":
        return "log_diet"
    elif intent == "log_workout":
        return "log_workout"
    elif intent == "analyze_workout":
        return "analyze_workout"
    else: # 'unclear' or any unexpected intent
        return "generate_response"

graph_builder.add_conditional_edges(
    "intent_router",
    route_intent,
    {
        "log_diet": "log_diet",
        "log_workout": "log_workout",
        "analyze_workout": "analyze_workout",
        "generate_response": "generate_response", # For 'unclear' intent
    },
)

# Add regular edges
graph_builder.add_edge("log_diet", "generate_response")
graph_builder.add_edge("log_workout", "generate_response")
graph_builder.add_edge("analyze_workout", "generate_response")

# Set graph termination
graph_builder.add_edge("generate_response", END)

# 4. Compilation
checkpointer = InMemoryCheckpointer()
final_app = graph_builder.compile(checkpointer=checkpointer)

if __name__ == "__main__":
    # Example Usage
    print("--- Running Example 1: Log Diet ---")
    inputs = {"user_input": "I ate an apple for snack (95 calories) and a chicken salad for lunch (350 calories)."}
    for s in final_app.stream(inputs):
        print(s)
    
    print("\n--- Running Example 2: Log Workout ---")
    inputs = {"user_input": "Just finished my workout: 3 sets of 12 reps bicep curls, 4 sets of 8 reps bench press."}
    for s in final_app.stream(inputs):
        print(s)

    print("\n--- Running Example 3: Analyze Workout (after logging) ---")
    # To see the analysis, we need to run the workout log again to accumulate data
    # Or manually set the state for analysis
    # Let's run the workout log again to simulate more data
    inputs_workout_2 = {"user_input": "Did 5 sets of bicep curls today, 10 reps each."}
    for s in final_app.stream(inputs_workout_2):
        print(s)

    # Now analyze
    inputs_analyze = {"user_input": "Can you analyze my workout progress?"}
    for s in final_app.stream(inputs_analyze):
        print(s)

    print("\n--- Running Example 4: Unclear Intent ---")
    inputs = {"user_input": "What's the weather like?"}
    for s in final_app.stream(inputs):
        print(s)

    print("\n--- Running Example 5: Empty Input ---")
    inputs = {"user_input": ""}
    for s in final_app.stream(inputs):
        print(s)
```

## Required Keys and Credentials
*   `OPENAI_API_KEY`: This environment variable must be set to your OpenAI API key for the `ChatOpenAI` model to function.

In [ ]:
Make code DFS an analyser 

In [ ]:
```python
# Phase 1: Graph Architecture Analysis & Strategy
# This graph implements a Router-Agent architecture.
# The 'intent_router' node acts as a supervisory agent, classifying the user's intent
# and dispatching the task to specialized "worker" nodes: 'log_diet', 'log_workout',
# or 'analyze_workout'. If the intent is unclear, it routes to a generic response.
# This architecture is chosen because the problem naturally breaks down into distinct,
# intent-driven tasks, each handled by a dedicated LLM-powered function,
# which is a common and effective pattern for building intelligent agents.

import json
from typing import TypedDict, List, Dict, Any, Union

from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
# from langchain_google_genai import ChatGoogleGenerativeAI # Uncomment if using Google GenAI

from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemoryCheckpointer

# TODO: Configure your LLM. For OpenAI, ensure OPENAI_API_KEY is set in your environment.
# For Google GenAI, ensure GOOGLE_API_KEY is set.
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
# llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0)

# Define Pydantic models for structured output from LLMs
class FoodItem(BaseModel):
    name: str = Field(description="Name of the food item.")
    calories: int = Field(description="Estimated calorie content of the food item.")
    meal_type: str = Field(description="Type of meal (e.g., breakfast, lunch, dinner, snack, drink, unknown).")

class WorkoutExercise(BaseModel):
    name: str = Field(description="Name of the exercise.")
    sets: int = Field(description="Number of sets performed.")
    reps: int = Field(description="Number of repetitions per set.")
    calories_burnt: int = Field(description="Estimated calories burnt during this exercise.")

# Phase 2: Python Code Generation

# State Definition (GraphState)
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        user_input: The initial input from the user.
        parsed_intent: The classified intent of the user's input ('log_diet', 'log_workout', 'analyze_workout', 'unclear').
        diet_log: A list of FoodItem objects (stored as dicts) representing logged food.
        workout_log: A list of WorkoutExercise objects (stored as dicts) representing logged workouts.
        daily_calories_consumed: Total calories consumed today.
        daily_calories_burnt: Total calories burnt today.
        weekly_workout_summary: A dictionary summarizing weekly workout stats (e.g., {'bicep_curls_total_sets': 10, 'bicep_goal_achieved': False}).
        analysis_result: The detailed analysis output from the 'analyze_workout' node.
        response: The final user-facing message.
    """
    user_input: str
    parsed_intent: str
    diet_log: List[Dict[str, Any]] # Stored as dicts for TypedDict compatibility
    workout_log: List[Dict[str, Any]] # Stored as dicts for TypedDict compatibility
    daily_calories_consumed: int
    daily_calories_burnt: int
    weekly_workout_summary: Dict[str, Any]
    analysis_result: str
    response: str

# Node Implementations (Python Functions)

def intent_router(state: GraphState) -> Dict[str, Any]:
    """
    Classifies the user's initial input to determine their intent.
    Uses an LLM for robust intent classification.
    """
    print("---INTENT ROUTER---")
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are an AI assistant that classifies user intent for a diet and workout tracker. Classify the following user input into one of these categories: 'log_diet', 'log_workout', 'analyze_workout', 'unclear'. Respond with only the category name."),
        ("user", "{user_input}")
    ])
    chain = prompt | llm | StrOutputParser()
    
    user_input = state.get("user_input", "")
    if not user_input:
        print("No user input provided, defaulting to 'unclear' intent.")
        return {"parsed_intent": "unclear"}

    intent = chain.invoke({"user_input": user_input})
    
    # Basic validation to ensure intent is one of the expected values
    valid_intents = ['log_diet', 'log_workout', 'analyze_workout', 'unclear']
    if intent not in valid_intents:
        print(f"LLM returned an invalid intent: '{intent}'. Defaulting to 'unclear'.")
        intent = "unclear"

    print(f"Parsed intent: {intent}")
    return {"parsed_intent": intent}

def log_diet(state: GraphState) -> Dict[str, Any]:
    """
    Processes user input related to diet. Extracts food items, estimates calories,
    and updates the diet log. Uses an LLM for structured data extraction.
    """
    print("---LOG DIET---")
    # Use with_structured_output for robust Pydantic model extraction
    structured_llm = llm.with_structured_output(List[FoodItem])
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are an AI assistant that extracts food items from user input. "
                   "Extract food items, their estimated calorie content, and meal type. "
                   "If no specific meal type is mentioned, use 'unknown'. "
                   "If no calories are mentioned, provide a reasonable estimate (e.g., apple 95, sandwich 300, soda 150). "
                   "If multiple items, list them all. If no food is mentioned, return an empty array. "),
        ("user", "{user_input}")
    ])
    chain = prompt | structured_llm
    
    user_input = state.get("user_input", "")
    extracted_food_items: List[FoodItem] = []
    try:
        extracted_food_items = chain.invoke({"user_input": user_input})
    except Exception as e:
        print(f"Error extracting food items: {e}. Returning empty list.")
        # Fallback to empty list if parsing fails or LLM returns invalid structure

    current_diet_log = state.get("diet_log", [])
    current_calories = state.get("daily_calories_consumed", 0)

    logged_items_names = []
    for item in extracted_food_items:
        current_diet_log.append(item.dict()) # Store as dict for TypedDict compatibility
        current_calories += item.calories
        logged_items_names.append(item.name)

    response_msg = ""
    if logged_items_names:
        response_msg = f"Logged {', '.join(logged_items_names)} as part of your diet."
    else:
        response_msg = "Could not identify any food items to log."
    
    response_msg += f" Calories consumed today: {current_calories}."

    print(f"Diet log updated. Response: {response_msg}")
    return {
        "diet_log": current_diet_log,
        "daily_calories_consumed": current_calories,
        "response": response_msg
    }

def log_workout(state: GraphState) -> Dict[str, Any]:
    """
    Processes user input related to workouts. Extracts exercise details,
    updates workout log and calorie burn. Uses an LLM for structured data extraction.
    """
    print("---LOG WORKOUT---")
    # Use with_structured_output for robust Pydantic model extraction
    structured_llm = llm.with_structured_output(List[WorkoutExercise])
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are an AI assistant that extracts workout exercises from user input. "
                   "Extract exercise name, sets, reps, and estimated calories burnt. "
                   "If no specific reps are mentioned, use 10. "
                   "If no calories are mentioned, estimate based on typical exercise intensity (e.g., 20-50 calories per set per exercise). "
                   "If multiple exercises, list them all. If no exercise is mentioned, return an empty array. "),
        ("user", "{user_input}")
    ])
    chain = prompt | structured_llm

    user_input = state.get("user_input", "")
    extracted_exercises: List[WorkoutExercise] = []
    try:
        extracted_exercises = chain.invoke({"user_input": user_input})
    except Exception as e:
        print(f"Error extracting workout exercises: {e}. Returning empty list.")
        # Fallback to empty list if parsing fails or LLM returns invalid structure

    current_workout_log = state.get("workout_log", [])
    current_calories_burnt = state.get("daily_calories_burnt", 0)
    current_weekly_summary = state.get("weekly_workout_summary", {})

    logged_exercises_names = []
    for exercise in extracted_exercises:
        current_workout_log.append(exercise.dict()) # Store as dict for TypedDict compatibility
        current_calories_burnt += exercise.calories_burnt
        logged_exercises_names.append(exercise.name)
        
        # Update weekly summary for analysis (example: bicep curls goal)
        exercise_name_key = exercise.name.lower().replace(' ', '_') + '_total_sets'
        current_weekly_summary[exercise_name_key] = current_weekly_summary.get(exercise_name_key, 0) + exercise.sets
        
    # Simple goal check for bicep training (example: 7 sets per week)
    if current_weekly_summary.get('bicep_curls_total_sets', 0) >= 7:
        current_weekly_summary['bicep_goal_achieved'] = True
    else:
        current_weekly_summary['bicep_goal_achieved'] = False

    response_msg = ""
    if logged_exercises_names:
        response_msg = f"Workout logged, {current_calories_burnt} calories burnt during the workout today."
    else:
        response_msg = "Could not identify any workouts to log."

    if current_weekly_summary.get('bicep_goal_achieved'):
        response_msg += " You have achieved the weekly goal for bicep training."

    print(f"Workout log updated. Response: {response_msg}")
    return {
        "workout_log": current_workout_log,
        "daily_calories_burnt": current_calories_burnt,
        "weekly_workout_summary": current_weekly_summary,
        "response": response_msg
    }

def analyze_workout(state: GraphState) -> Dict[str, Any]:
    """
    Provides insights and recommendations based on the aggregated weekly_workout_summary.
    Uses an LLM to generate a comprehensive analysis.
    """
    print("---ANALYZE WORKOUT---")
    weekly_summary = state.get("weekly_workout_summary", {})
    workout_log = state.get("workout_log", [])
    
    prompt_template = ChatPromptTemplate.from_messages([
        ("system", "You are an AI fitness coach. Analyze the provided workout data and provide insights, "
                   "recommendations, and encouragement. Focus on progress, areas for improvement, "
                   "and general fitness advice. If no data is available, state that clearly. "
                   "Consider the following data:\n"
                   "Weekly Summary: {weekly_summary}\n"
                   "Full Workout Log: {workout_log}\n"
                   "Daily Calories Burnt: {daily_calories_burnt}\n"
                   "User Input: {user_input}\n"
                   "Provide a concise, helpful analysis."),
        ("user", "Analyze my workout data.")
    ])
    
    chain = prompt_template | llm | StrOutputParser()

    analysis_output = chain.invoke({
        "weekly_summary": json.dumps(weekly_summary, indent=2),
        "workout_log": json.dumps(workout_log, indent=2),
        "daily_calories_burnt": state.get("daily_calories_burnt", 0),
        "user_input": state.get("user_input", "")
    })

    print(f"Workout analysis generated. Response: {analysis_output}")
    return {
        "analysis_result": analysis_output,
        "response": analysis_output # For simplicity, analysis result is the response
    }

def generate_response(state: GraphState) -> Dict[str, Any]:
    """
    Generates the final user-facing response. Prioritizes responses set by
    action nodes, otherwise provides a default message.
    """
    print("---GENERATE RESPONSE---")
    # If an action node has already set a specific response, use it.
    # Otherwise, provide a default message for unclear intents.
    if state.get("response"):
        final_response = state["response"]
    else:
        final_response = "I'm not sure how to help with that. Please tell me if you want to log diet, log a workout, or get a workout analysis."
    
    print(f"Final response: {final_response}")
    return {"response": final_response}

# Graph Construction (StatefulGraph)
graph_builder = StateGraph(GraphState)

# Add nodes
graph_builder.add_node("intent_router", intent_router)
graph_builder.add_node("log_diet", log_diet)
graph_builder.add_node("log_workout", log_workout)
graph_builder.add_node("analyze_workout", analyze_workout)
graph_builder.add_node("generate_response", generate_response)

# Set entry point
graph_builder.add_edge(START, "intent_router")

# Define conditional routing for intent_router
def route_intent(state: GraphState) -> str:
    """
    Routes the graph based on the parsed intent.
    """
    intent = state.get("parsed_intent")
    print(f"Routing based on intent: {intent}")
    if intent == "log_diet":
        return "log_diet"
    elif intent == "log_workout":
        return "log_workout"
    elif intent == "analyze_workout":
        return "analyze_workout"
    else: # 'unclear' or any unexpected intent
        return "generate_response"

graph_builder.add_conditional_edges(
    "intent_router",
    route_intent,
    {
        "log_diet": "log_diet",
        "log_workout": "log_workout",
        "analyze_workout": "analyze_workout",
        "generate_response": "generate_response", # For 'unclear' intent
    },
)

# Add regular edges from action nodes to generate_response
graph_builder.add_edge("log_diet", "generate_response")
graph_builder.add_edge("log_workout", "generate_response")
graph_builder.add_edge("analyze_workout", "generate_response")

# Set graph termination
graph_builder.add_edge("generate_response", END)

# Compile the graph
checkpointer = InMemoryCheckpointer()
final_app = graph_builder.compile(checkpointer=checkpointer)

# Example Usage (Optional, for testing)
if __name__ == "__main__":
    print("--- Running LangGraph Application ---")

    # Test Case 1: Log Diet
    print("\n--- Test Case 1: Log Diet ---")
    inputs_diet = {"user_input": "I ate an apple for 95 calories and a sandwich for 300 calories for lunch."}
    # Using a fixed thread_id for demonstration purposes to accumulate state
    thread_id = "user123" 
    for s in final_app.stream(inputs_diet, config={"configurable": {"thread_id": thread_id}}):
        print(s)
    print(f"Final State after diet log: {final_app.get_state(thread_id=thread_id).values}")

    # Test Case 2: Log Workout
    print("\n--- Test Case 2: Log Workout ---")
    inputs_workout = {"user_input": "I did 3 sets of 10 reps of bicep curls and 4 sets of 8 reps of squats, burnt 200 calories."}
    for s in final_app.stream(inputs_workout, config={"configurable": {"thread_id": thread_id}}):
        print(s)
    print(f"Final State after workout log: {final_app.get_state(thread_id=thread_id).values}")

    # Test Case 3: Analyze Workout (after logging some workouts)
    print("\n--- Test Case 3: Analyze Workout ---")
    inputs_analyze = {"user_input": "How was my workout performance this week?"}
    for s in final_app.stream(inputs_analyze, config={"configurable": {"thread_id": thread_id}}):
        print(s)
    print(f"Final State after workout analysis: {final_app.get_state(thread_id=thread_id).values}")

    # Test Case 4: Unclear Intent
    print("\n--- Test Case 4: Unclear Intent ---")
    inputs_unclear = {"user_input": "Tell me a joke."}
    # Using a new thread_id for a fresh start
    thread_id_unclear = "user_unclear"
    for s in final_app.stream(inputs_unclear, config={"configurable": {"thread_id": thread_id_unclear}}):
        print(s)
    print(f"Final State after unclear intent: {final_app.get_state(thread_id=thread_id_unclear).values}")

    # Test Case 5: Log more bicep curls to hit goal (continuing from original thread)
    print("\n--- Test Case 5: Log more bicep curls to hit goal ---")
    inputs_bicep_goal = {"user_input": "Did 2 sets of bicep curls today."}
    for s in final_app.stream(inputs_bicep_goal, config={"configurable": {"thread_id": thread_id}}):
        print(s)
    print(f"Final State after hitting bicep goal: {final_app.get_state(thread_id=thread_id).values}")
    
    # Test Case 6: Analyze Workout again to see goal achievement (continuing from original thread)
    print("\n--- Test Case 6: Analyze Workout again to see goal achievement ---")
    inputs_analyze_again = {"user_input": "What about my bicep goal?"}
    for s in final_app.stream(inputs_analyze_again, config={"configurable": {"thread_id": thread_id}}):
        print(s)
    print(f"Final State after re-analysis: {final_app.get_state(thread_id=thread_id).values}")

```

## Required Keys and Credentials

For the generated Python code to execute successfully, you will need to set up the following environment variables:

*   **`OPENAI_API_KEY`**: Required if you are using `ChatOpenAI` as your LLM.
    *   You can obtain this key from the OpenAI platform.

If you choose to use `ChatGoogleGenerativeAI` (by uncommenting the relevant lines), you would need:

*   **`GOOGLE_API_KEY`**: Required for Google's Generative AI models (e.g., Gemini).
    *   You can obtain this key from the Google Cloud Console or Google AI Studio.

**How to set environment variables:**

Before running the script, set the environment variable in your terminal:

```bash
export OPENAI_API_KEY="your_openai_api_key_here"
# Or for Google:
# export GOOGLE_API_KEY="your_google_api_key_here"
```